<a href="https://colab.research.google.com/github/treyra/CS155_miniproject3/blob/main/Indecision_Trees_Notebook_MiniProject_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Pre-processing



In [2]:
import numpy as np
from tqdm.notebook import tqdm

In [2]:
# read in the data
# Import Data
data = np.genfromtxt('https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project3/data/shakespeare.txt',delimiter='\t',dtype=None,encoding='ISO-8859-1')
data[-15]

'154'

In [ ]:
# make all words lower case
for i in range(len(data)):
  data[i]=data[i].lower()

In [ ]:
# remove space at beginning of line
for i in range(len(data)):
  while data[i][0] == ' ':
    line = list(data[i])
    data[i] = "".join(line[1:len(line)])

In [ ]:
# remove \n
for i in range(len(data)):
        data[i] = data[i].replace('\n', '')

In [ ]:
# remove punctuation
to_remove = ['.', ',', ';', ':', '?', '!', '-', "'"]
for k in range(len(to_remove)):
  for i in range(len(data)):
    data[i] = ''.join([j for j in data[i] if j not in to_remove])

In [ ]:
# remove empty lines
# find indices to remove
remove_idx = []
for i in range(len(data)):
  if data[i] == '\n':
    remove_idx.append(i)
  if data[i] == '':
    remove_idx.append(i)
# remove
data_clean = []
for k in range(len(data)):
  if k not in remove_idx:
    data_clean.append(data[k])
data = data_clean

In [ ]:
# remove sonnets which are too long or too short
# always 14 lines long
# check how long each sonnet is - should be 14!
digit_idxs = []
for i in range(len(data)):
  if data[i].isdigit():
    digit_idxs.append(i)
#print(digit_idxs)
# get difference
length = np.diff(np.array(digit_idxs))
#print(length)
# remove the sonnet with too many or too few lines
remove_nbrs = dict()
for i in range(len(length)):
  if length[i]!=15:
    remove_nbrs.update({data[digit_idxs[i]]:length[i]})
# remove the sonnets with the numbers in remove_numbers
remove_idx = []
for i in range(len(data)):
  if data[i] in remove_nbrs:
    print(data[i])
    for k in range(0,remove_nbrs[data[i]]):
      remove_idx.append(i+k)
# remove
data_clean = []
for k in range(len(data)):
  if k not in remove_idx:
    data_clean.append(data[k])
data = data_clean

99
126


In [ ]:
# remove numbers
for i in range(len(data)):
  data[i] = ''.join([j for j in data[i] if not j.isdigit()])
# remove created empty lines again
# find indices to remove
remove_idx = []
for i in range(len(data)):
  if data[i] == '\n':
    remove_idx.append(i)
  if data[i] == '':
    remove_idx.append(i)
# remove
data_clean = []
for k in range(len(data)):
  if k not in remove_idx:
    data_clean.append(data[k])
data = data_clean

In [ ]:
# this is the data per line
data_by_line = data

In [ ]:
# get data per sonnet
# split up in sonnets
nr_sonnets = len(data)/14
data_by_sonnet = [None] * int(nr_sonnets)
i = 0
for k in range(0, len(data), 14):
  cString = ''
  for idx in range(14):
    #print(idx)
    if idx != 0:
      try:
        cString = cString + ' ' + data[k + idx]
      except:
        import pdb; pdb.set_trace()
    else: 
      cString = data[k]
  data_by_sonnet[i] = cString
  i+=1

In [ ]:
# make new file for data per line
with open('/content/drive/My Drive/CS155_miniproject_3/data_line.txt', 'w') as f:
  for item in data_by_line:
        f.write("%s\n" % item)

In [ ]:
# make new file for data per sonnet
with open('/content/drive/My Drive/CS155_miniproject_3/data_sonnet.txt', 'w') as f:
  for item in data_by_sonnet:
        f.write("%s\n" % item)

We can also load the pre-processed data directly as:

In [3]:
# load the data - for now from drive until we have repo
# open existing text file
data_sonnet_raw = np.genfromtxt('https://raw.githubusercontent.com/treyra/CS155_miniproject3/main/data/data_sonnet.txt',delimiter='\t',dtype=None,encoding='ISO-8859-1')
data_line = np.genfromtxt('https://raw.githubusercontent.com/treyra/CS155_miniproject3/main/data/data_line.txt',delimiter='\t',dtype=None,encoding='ISO-8859-1')

#HMM Model

Based off of HW 6's solution code

In [4]:
class HiddenMarkovModel:
    '''
    Class implementation of Hidden Markov Models.
    '''

    def __init__(self, A, O):
        '''
        Initializes an HMM. Assumes the following:
            - States and observations are integers starting from 0. 
            - There is a start state (see notes on A_start below). There
              is no integer associated with the start state, only
              probabilities in the vector A_start.
            - There is no end state. 
        Arguments:
            A:          Transition matrix with dimensions L x L.
                        The (i, j)^th element is the probability of
                        transitioning from state i to state j. Note that
                        this does not include the starting probabilities.
            O:          Observation matrix with dimensions L x D.
                        The (i, j)^th element is the probability of
                        emitting observation j given state i.
        Parameters:
            L:          Number of states.
            D:          Number of observations.
            
            A:          The transition matrix.
            
            O:          The observation matrix.
            
            A_start:    Starting transition probabilities. The i^th element
                        is the probability of transitioning from the start
                        state to state i. For simplicity, we assume that
                        this distribution is uniform.
        '''

        self.L = len(A)
        self.D = len(O[0])
        self.A = np.array(A)
        self.O = np.array(O)
        self.A_start = [1. / self.L for _ in range(self.L)]


    def viterbi(self, x):
        '''
        Uses the Viterbi algorithm to find the max probability state 
        sequence corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            max_seq:    Output sequence corresponding to x with the highest
                        probability.
        '''

        M = len(x)      # Length of sequence.

        # The (i, j)^th elements of probs and seqs are the max probability
        # of the prefix of length i ending in state j and the prefix
        # that gives this probability, respectively.
        #
        # For instance, probs[1][0] is the probability of the prefix of
        # length 1 ending in state 0.
        probs = [[0. for _ in range(self.L)] for _ in range(M + 1)]
        seqs = [['' for _ in range(self.L)] for _ in range(M + 1)]

        # Calculate initial prefixes and probabilities.
        for curr in range(self.L):
            probs[1][curr] = self.A_start[curr] * self.O[curr][x[0]]
            seqs[1][curr] = str(curr)

        # Calculate best prefixes and probabilities throughout sequence.
        for t in range(2, M + 1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                max_prob = float("-inf")
                max_prefix = ''

                # Iterate over all possible previous states to find one
                # that would maximize the probability of the current state.
                for prev in range(self.L):
                    curr_prob = probs[t - 1][prev] \
                                * self.A[prev][curr] \
                                * self.O[curr][x[t - 1]]

                    # Continually update max probability and prefix.
                    if curr_prob >= max_prob:
                        max_prob = curr_prob
                        max_prefix = seqs[t - 1][prev]

                # Store the max probability and prefix.
                probs[t][curr] = max_prob
                seqs[t][curr] = max_prefix + str(curr)

        # Find the index of the max probability of a sequence ending in x^M
        # and the corresponding output sequence.
        max_i = max(enumerate(probs[-1]), key=lambda x: x[1])[0]
        max_seq = seqs[-1][max_i]

        return max_seq


    def forward(self, x, normalize=False):
        '''
        Uses the forward algorithm to calculate the alpha probability
        vectors corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.
        Returns:
            alphas:     Vector of alphas.
                        The (i, j)^th element of alphas is alpha_j(i),
                        i.e. the probability of observing prefix x^1:i
                        and state y^i = j.
                        e.g. alphas[1][0] corresponds to the probability
                        of observing x^1:1, i.e. the first observation,
                        given that y^1 = 0, i.e. the first state is 0.
        '''

        M = len(x)      # Length of sequence.
        alphas = [[0. for _ in range(self.L)] for _ in range(M + 1)]

        # Note that alpha_j(0) is already correct for all j's.
        # Calculate alpha_j(1) for all j's.
        for curr in range(self.L):
            alphas[1][curr] = self.A_start[curr] * self.O[curr][x[0]]

        # Calculate alphas throughout sequence.
        for t in range(1, M):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible previous states to accumulate
                # the probabilities of all paths from the start state to
                # the current state.
                for prev in range(self.L):
                    prob += alphas[t][prev] \
                            * self.A[prev][curr] \
                            * self.O[curr][x[t]]

                # Store the accumulated probability.
                alphas[t + 1][curr] = prob

            if normalize:
                norm = sum(alphas[t + 1])
                for curr in range(self.L):
                    alphas[t + 1][curr] /= norm

        return alphas


    def backward(self, x, normalize=False):
        '''
        Uses the backward algorithm to calculate the beta probability
        vectors corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.
        Returns:
            betas:      Vector of betas.
                        The (i, j)^th element of betas is beta_j(i), i.e.
                        the probability of observing prefix x^(i+1):M and
                        state y^i = j.
                        e.g. betas[M][0] corresponds to the probability
                        of observing x^M+1:M, i.e. no observations,
                        given that y^M = 0, i.e. the last state is 0.
        '''

        M = len(x)      # Length of sequence.
        betas = [[0. for _ in range(self.L)] for _ in range(M + 1)]

        # Initialize initial betas.
        for curr in range(self.L):
            betas[-1][curr] = 1

        # Calculate betas throughout sequence.
        for t in range(-1, -M - 1, -1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible next states to accumulate
                # the probabilities of all paths from the end state to
                # the current state.
                for nxt in range(self.L):
                    if t == -M:
                        prob += betas[t][nxt] \
                                * self.A_start[nxt] \
                                * self.O[nxt][x[t]]

                    else:
                        prob += betas[t][nxt] \
                                * self.A[curr][nxt] \
                                * self.O[nxt][x[t]]

                # Store the accumulated probability.
                betas[t - 1][curr] = prob

            if normalize:
                norm = sum(betas[t - 1])
                for curr in range(self.L):
                    betas[t - 1][curr] /= norm

        return betas


    def supervised_learning(self, X, Y):
        '''
        Trains the HMM using the Maximum Likelihood closed form solutions
        for the transition and observation matrices on a labeled
        datset (X, Y). Note that this method does not return anything, but
        instead updates the attributes of the HMM object.
        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of variable length, consisting of integers 
                        ranging from 0 to D - 1. In other words, a list of
                        lists.
            Y:          A dataset consisting of state sequences in the form
                        of lists of variable length, consisting of integers 
                        ranging from 0 to L - 1. In other words, a list of
                        lists.
                        Note that the elements in X line up with those in Y.
        '''

        # Calculate each element of A using the M-step formulas.
        for curr in range(self.L):
            for nxt in range(self.L):
                num = 0.
                den = 0.

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)
        
                    num += len([1 for i in range(M - 1) \
                                if y[i] == curr and y[i + 1] == nxt])
                    den += len([1 for i in range(M - 1) if y[i] == curr])

                self.A[curr][nxt] = num / den

        # Calculate each element of O using the M-step formulas.
        for curr in range(self.L):
            for xt in range(self.D):
                num = 0.
                den = 0.

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)
        
                    num += len([1 for i in range(M) \
                                if y[i] == curr and x[i] == xt])
                    den += len([1 for i in range(M) if y[i] == curr])

                self.O[curr][xt] = num / den


    def unsupervised_learning(self, X, N_iters):
        '''
        Trains the HMM using the Baum-Welch algorithm on an unlabeled
        datset X. Note that this method does not return anything, but
        instead updates the attributes of the HMM object.
        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of length M, consisting of integers ranging
                        from 0 to D - 1. In other words, a list of lists.
            N_iters:    The number of iterations to train on.
        '''

        # Note that a comment starting with 'E' refers to the fact that
        # the code under the comment is part of the E-step.

        # Similarly, a comment starting with 'M' refers to the fact that
        # the code under the comment is part of the M-step.

        for iteration in tqdm(range(1, N_iters + 1)):
            if iteration % 10 == 0:
                print("Iteration: " + str(iteration))

            # Numerator and denominator for the update terms of A and O.
            A_num = [[0. for i in range(self.L)] for j in range(self.L)]
            O_num = [[0. for i in range(self.D)] for j in range(self.L)]
            A_den = [0. for i in range(self.L)]
            O_den = [0. for i in range(self.L)]

            # For each input sequence:
            for x in X:
                M = len(x)
                # Compute the alpha and beta probability vectors.
                alphas = self.forward(x, normalize=True)
                betas = self.backward(x, normalize=True)

                # E: Update the expected observation probabilities for a
                # given (x, y).
                # The i^th index is P(y^t = i, x).
                for t in range(1, M + 1):
                    P_curr = [0. for _ in range(self.L)]
                    
                    for curr in range(self.L):
                        P_curr[curr] = alphas[t][curr] * betas[t][curr]

                    # Normalize the probabilities.
                    norm = sum(P_curr)
                    for curr in range(len(P_curr)):
                        P_curr[curr] /= norm

                    for curr in range(self.L):
                        if t != M:
                            A_den[curr] += P_curr[curr]
                        O_den[curr] += P_curr[curr]
                        O_num[curr][x[t - 1]] += P_curr[curr]

                # E: Update the expectedP(y^j = a, y^j+1 = b, x) for given (x, y)
                for t in range(1, M):
                    P_curr_nxt = [[0. for _ in range(self.L)] for _ in range(self.L)]

                    for curr in range(self.L):
                        for nxt in range(self.L):
                            P_curr_nxt[curr][nxt] = alphas[t][curr] \
                                                    * self.A[curr][nxt] \
                                                    * self.O[nxt][x[t]] \
                                                    * betas[t + 1][nxt]

                    # Normalize:
                    norm = 0
                    for lst in P_curr_nxt:
                        norm += sum(lst)
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            P_curr_nxt[curr][nxt] /= norm

                    # Update A_num
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            A_num[curr][nxt] += P_curr_nxt[curr][nxt]

            for curr in range(self.L):
                for nxt in range(self.L):
                    self.A[curr][nxt] = A_num[curr][nxt] / A_den[curr]

            for curr in range(self.L):
                for xt in range(self.D):
                    self.O[curr][xt] = O_num[curr][xt] / O_den[curr]

    def generate_emission(self, M):
        '''
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random. 
        Arguments:
            M:          Length of the emission to generate.
        Returns:
            emission:   The randomly generated emission as a list.
            states:     The randomly generated states as a list.
        '''

        emission = []
        state = random.choice(range(self.L))
        states = []

        for t in range(M):
            # Append state.
            states.append(state)

            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1
            emission.append(next_obs)

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= self.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state

        return emission, states


    def probability_alphas(self, x):
        '''
        Finds the maximum probability of a given input sequence using
        the forward algorithm.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            prob:       Total probability that x can occur.
        '''

        # Calculate alpha vectors.
        alphas = self.forward(x)

        # alpha_j(M) gives the probability that the output sequence ends
        # in j. Summing this value over all possible states j gives the
        # total probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum(alphas[-1])
        return prob


    def probability_betas(self, x):
        '''
        Finds the maximum probability of a given input sequence using
        the backward algorithm.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            prob:       Total probability that x can occur.
        '''

        betas = self.backward(x)

        # beta_j(0) gives the probability of the output sequence. Summing
        # this over all states and then normalizing gives the total
        # probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum([betas[1][k] * self.A_start[k] * self.O[k][x[0]] \
            for k in range(self.L)])

        return prob


def supervised_HMM(X, Y):
    '''
    Helper function to train a supervised HMM. The function determines the
    number of unique states and observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for supervised learning.
    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.
        Y:          A dataset consisting of state sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to L - 1. In other words, a list of lists.
                    Note that the elements in X line up with those in Y.
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Make a set of states.
    states = set()
    for y in Y:
        states |= set(y)
    
    # Compute L and D.
    L = len(states)
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm
    
    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with labeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.supervised_learning(X, Y)

    return HMM


def unsupervised_HMM(X, n_states, N_iters,rng=np.random.RandomState(1)):
    '''
    Helper function to train an unsupervised HMM. The function determines the
    number of unique observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for unsupervised learing.
    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.
        n_states:   Number of hidden states to use in training.
        
        N_iters:    The number of iterations to train on.
        rng:        The random number generator for reproducible result.
                    Default to RandomState(1).
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)
    
    # Compute L and D.
    L = n_states
    D = len(observations)

    # Randomly initialize and normalize matrices A.
    A = [[rng.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm
    
    # Randomly initialize and normalize matrix O.
    O = [[rng.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with unlabeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.unsupervised_learning(X, N_iters)

    return HMM

In [5]:
########################################
# CS/CNS/EE 155 2018
# Problem Set 6
#
# Author:       Andrew Kang
# Description:  Set 6 HMM helper
########################################

import re
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation


####################
# WORDCLOUD FUNCTIONS
####################

def mask():
    # Parameters.
    r = 128
    d = 2 * r + 1

    # Get points in a circle.
    y, x = np.ogrid[-r:d-r, -r:d-r]
    circle = (x**2 + y**2 <= r**2)

    # Create mask.
    mask = 255 * np.ones((d, d), dtype=np.uint8)
    mask[circle] = 0

    return mask

def text_to_wordcloud(text, max_words=50, title='', show=True):
    plt.close('all')

    # Generate a wordcloud image.
    wordcloud = WordCloud(random_state=0,
                          max_words=max_words,
                          background_color='white',
                          mask=mask()).generate(text)

    # Show the image.
    if show:
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(title, fontsize=24)
        plt.show()

    return wordcloud

def states_to_wordclouds(hmm, obs_map, max_words=50, show=True):
    # Initialize.
    M = 100000
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = []

    # Generate a large emission.
    emission, states = hmm.generate_emission(M)

    # For each state, get a list of observations that have been emitted
    # from that state.
    obs_count = []
    for i in range(n_states):
        obs_lst = np.array(emission)[np.where(np.array(states) == i)[0]]
        obs_count.append(obs_lst)

    # For each state, convert it into a wordcloud.
    for i in range(n_states):
        obs_lst = obs_count[i]
        sentence = [obs_map_r[j] for j in obs_lst]
        sentence_str = ' '.join(sentence)

        wordclouds.append(text_to_wordcloud(sentence_str, max_words=max_words, title='State %d' % i, show=show))

    return wordclouds


####################
# HMM FUNCTIONS
####################

def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

def sample_sentence(hmm, obs_map, n_words=100):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)

    # Sample and convert sentence.
    emission, states = hmm.generate_emission(n_words)
    sentence = [obs_map_r[i] for i in emission]

    return ' '.join(sentence).capitalize() + '...'


####################
# HMM VISUALIZATION FUNCTIONS
####################

def visualize_sparsities(hmm, O_max_cols=50, O_vmax=0.1):
    plt.close('all')
    plt.set_cmap('viridis')

    # Visualize sparsity of A.
    plt.imshow(hmm.A, vmax=1.0)
    plt.colorbar()
    plt.title('Sparsity of A matrix')
    plt.show()

    # Visualize parsity of O.
    plt.imshow(np.array(hmm.O)[:, :O_max_cols], vmax=O_vmax, aspect='auto')
    plt.colorbar()
    plt.title('Sparsity of O matrix')
    plt.show()


####################
# HMM ANIMATION FUNCTIONS
####################

def animate_emission(hmm, obs_map, M=8, height=12, width=12, delay=1):
    # Parameters.
    lim = 1200
    text_x_offset = 40
    text_y_offset = 80
    x_offset = 580
    y_offset = 520
    R = 420
    r = 100
    arrow_size = 20
    arrow_p1 = 0.03
    arrow_p2 = 0.02
    arrow_p3 = 0.06
    
    # Initialize.
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = states_to_wordclouds(hmm, obs_map, max_words=20, show=False)

    # Initialize plot.    
    fig, ax = plt.subplots()
    fig.set_figheight(height)
    fig.set_figwidth(width)
    ax.grid('off')
    plt.axis('off')
    ax.set_xlim([0, lim])
    ax.set_ylim([0, lim])

    # Plot each wordcloud.
    for i, wordcloud in enumerate(wordclouds):
        x = x_offset + int(R * np.cos(np.pi * 2 * i / n_states))
        y = y_offset + int(R * np.sin(np.pi * 2 * i / n_states))
        ax.imshow(wordcloud.to_array(), extent=(x - r, x + r, y - r, y + r), aspect='auto', zorder=-1)

    # Initialize text.
    text = ax.text(text_x_offset, lim - text_y_offset, '', fontsize=24)
        
    # Make the arrows.
    zorder_mult = n_states ** 2 * 100
    arrows = []
    for i in range(n_states):
        row = []
        for j in range(n_states):
            # Arrow coordinates.
            x_i = x_offset + R * np.cos(np.pi * 2 * i / n_states)
            y_i = y_offset + R * np.sin(np.pi * 2 * i / n_states)
            x_j = x_offset + R * np.cos(np.pi * 2 * j / n_states)
            y_j = y_offset + R * np.sin(np.pi * 2 * j / n_states)
            
            dx = x_j - x_i
            dy = y_j - y_i
            d = np.sqrt(dx**2 + dy**2)

            if i != j:
                arrow = ax.arrow(x_i + (r/d + arrow_p1) * dx + arrow_p2 * dy,
                                 y_i + (r/d + arrow_p1) * dy + arrow_p2 * dx,
                                 (1 - 2 * r/d - arrow_p3) * dx,
                                 (1 - 2 * r/d - arrow_p3) * dy,
                                 color=(1 - hmm.A[i][j], ) * 3,
                                 head_width=arrow_size, head_length=arrow_size,
                                 zorder=int(hmm.A[i][j] * zorder_mult))
            else:
                arrow = ax.arrow(x_i, y_i, 0, 0,
                                 color=(1 - hmm.A[i][j], ) * 3,
                                 head_width=arrow_size, head_length=arrow_size,
                                 zorder=int(hmm.A[i][j] * zorder_mult))

            row.append(arrow)
        arrows.append(row)

    emission, states = hmm.generate_emission(M)

    def animate(i):
        if i >= delay:
            i -= delay

            if i == 0:
                arrows[states[0]][states[0]].set_color('red')
            elif i == 1:
                arrows[states[0]][states[0]].set_color((1 - hmm.A[states[0]][states[0]], ) * 3)
                arrows[states[i - 1]][states[i]].set_color('red')
            else:
                arrows[states[i - 2]][states[i - 1]].set_color((1 - hmm.A[states[i - 2]][states[i - 1]], ) * 3)
                arrows[states[i - 1]][states[i]].set_color('red')

            # Set text.
            text.set_text(' '.join([obs_map_r[e] for e in emission][:i+1]).capitalize())

            return arrows + [text]

    # Animate!
    print('\nAnimating...')
    anim = FuncAnimation(fig, animate, frames=M+delay, interval=1000)

    return anim

Making the syllable dictionary

In [6]:
!wget https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project3/data/Syllable_dictionary.txt

--2021-03-09 02:21:52--  https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project3/data/Syllable_dictionary.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33379 (33K) [text/plain]
Saving to: ‘Syllable_dictionary.txt’

Syllable_dictionary 100%[===================>]  32.60K  --.-KB/s    in 0.001s  

2021-03-09 02:21:52 (54.0 MB/s) - ‘Syllable_dictionary.txt’ saved [33379/33379]



In [7]:
import re
with open('Syllable_dictionary.txt', "r") as f:
  syl_data = f.readlines()
  #dict_exp = dict(re.findall(r'(\S+)\s+(.+)', f.read()))

In [8]:
# remove all ' or " and \n
# remove punctuation
to_remove = ['-', "'"]
for k in range(len(to_remove)):
  for i in range(len(syl_data)):
    syl_data[i] = ''.join([j for j in syl_data[i] if j not in to_remove])
#print(syl_data[0])
for i in range(len(syl_data)):
  syl_data[i] = syl_data[i].replace('\n', '')
#print(syl_data[0])
# split the lines into words
for i in range(len(syl_data)):
  syl_data[i] = re.findall(r"[\w']+|[.,?;]", syl_data[i])
#print(syl_data[0])
# make syl_data a dictionary
syl_dict = dict()
for line in syl_data:
  #print(line)
  #import pdb; pdb.set_trace()
  syl_dict.update({line[0]: [line[i] for i in range(1, len(line))]})

# Generate Random Sonnet

First input data and train HMM

Modify raw text data into data we can use for our syllable dictionary

In [22]:
data_sonnet = [[]]
for i,sonnet in enumerate(data_sonnet_raw):
    data_sonnet[i] = sonnet.split()
    if i != len(data_sonnet_raw)-1:
        data_sonnet.append([])

In [23]:
data_sonnet

[['from',
  'fairest',
  'creatures',
  'we',
  'desire',
  'increase',
  'that',
  'thereby',
  'beautys',
  'rose',
  'might',
  'never',
  'die',
  'but',
  'as',
  'the',
  'riper',
  'should',
  'by',
  'time',
  'decease',
  'his',
  'tender',
  'heir',
  'might',
  'bear',
  'his',
  'memory',
  'but',
  'thou',
  'contracted',
  'to',
  'thine',
  'own',
  'bright',
  'eyes',
  'feedst',
  'thy',
  'lights',
  'flame',
  'with',
  'selfsubstantial',
  'fuel',
  'making',
  'a',
  'famine',
  'where',
  'abundance',
  'lies',
  'thy',
  'self',
  'thy',
  'foe',
  'to',
  'thy',
  'sweet',
  'self',
  'too',
  'cruel',
  'thou',
  'that',
  'art',
  'now',
  'the',
  'worlds',
  'fresh',
  'ornament',
  'and',
  'only',
  'herald',
  'to',
  'the',
  'gaudy',
  'spring',
  'within',
  'thine',
  'own',
  'bud',
  'buriest',
  'thy',
  'content',
  'and',
  'tender',
  'churl',
  'makst',
  'waste',
  'in',
  'niggarding',
  'pity',
  'the',
  'world',
  'or',
  'else',
  'this',

In [24]:
# create obs map
count = 0
obs = []
obs_map = {}

for line in data_sonnet:
  obs_line = []
  for word in line:
    if word not in obs_map:
      # add new words to the map
      obs_map[word] = count
      count += 1
    # add the encoded words to the line map
    obs_line.append(obs_map[word])
      
    # add the encoded sequence
  obs.append(obs_line)

In [16]:
data_sonnet

[[]]

In [25]:
# train a HMM (only a few epochs)
hmm8 = unsupervised_HMM(obs, 8, 20)

Iteration: 10
Iteration: 20



In [30]:
print(obs_map)

{'from': 0, 'fairest': 1, 'creatures': 2, 'we': 3, 'desire': 4, 'increase': 5, 'that': 6, 'thereby': 7, 'beautys': 8, 'rose': 9, 'might': 10, 'never': 11, 'die': 12, 'but': 13, 'as': 14, 'the': 15, 'riper': 16, 'should': 17, 'by': 18, 'time': 19, 'decease': 20, 'his': 21, 'tender': 22, 'heir': 23, 'bear': 24, 'memory': 25, 'thou': 26, 'contracted': 27, 'to': 28, 'thine': 29, 'own': 30, 'bright': 31, 'eyes': 32, 'feedst': 33, 'thy': 34, 'lights': 35, 'flame': 36, 'with': 37, 'selfsubstantial': 38, 'fuel': 39, 'making': 40, 'a': 41, 'famine': 42, 'where': 43, 'abundance': 44, 'lies': 45, 'self': 46, 'foe': 47, 'sweet': 48, 'too': 49, 'cruel': 50, 'art': 51, 'now': 52, 'worlds': 53, 'fresh': 54, 'ornament': 55, 'and': 56, 'only': 57, 'herald': 58, 'gaudy': 59, 'spring': 60, 'within': 61, 'bud': 62, 'buriest': 63, 'content': 64, 'churl': 65, 'makst': 66, 'waste': 67, 'in': 68, 'niggarding': 69, 'pity': 70, 'world': 71, 'or': 72, 'else': 73, 'this': 74, 'glutton': 75, 'be': 76, 'eat': 77, '

In [33]:
# now generate a random sonnet with 14 lines and 10 syllables each
emission = []
states = []
L = np.array(hmm8.L)
D = np.array(hmm8.D)  
O = np.array(hmm8.O)
A = np.array(hmm8.A)
obs_map_r = obs_map_reverser(obs_map)

# make sure we have 14 lines
line_syls = []
for i in range(0,14):
  # each line with 10 syllables
  emission = []
  states = []
  k = 0
  # Initial state
  states.append(np.random.randint(0,L))
  emission.append(np.random.choice(D,None, True, O[states[0],:]))
  syl_count= syl_dict[obs_map_r[emission[0]]]
  if len(syl_count)>1:
    # check to use the one not at the end or just randomly
    not_end_count = [x for x in syl_count if not x.startswith('E')]
    syl_count = not_end_count
    # if syl_count[0][0]!='E':
    #   syl_count = syl_count[0][0]
  try:
    k+=int(syl_count[0])
  except:
    import pdb; pdb.set_trace()
  j=0
  #print(k)
  while k<10:
    #import pdb; pdb.set_trace()
    j+=1
    # generate next word
    states.append(np.random.choice(L,None, True, A[states[j-1],:]))
    emission.append(np.random.choice(D,None, True, O[states[j],:]))
    syl_count= syl_dict[obs_map_r[emission[j]]]
    #print(obs_map_r[emission[j]])
    if len(syl_count)>1:
      end_count = [x for x in syl_count if x.startswith('E')]
      #import pdb; pdb.set_trace()
      if len(end_count) >= 1:
        if k + int(end_count[0][-1]) > 10:
          # if equal to 10 pick this word and save sentence
          if k + int(end_count[0][-1]) == 10:
            syl_count=int(end_count[0][-1])
        # check to use the correct one
        else:
          for s in syl_count:
            if s[0]!='E':
              syl_count = int(s)
      else:
        syl_count = int(syl_count[0])
    else:
      syl_count = int(syl_count[0])
    if k+syl_count>10:
      # regernerate
      while k+syl_count>10:
        states[j] = (np.random.choice(L,None, True, A[states[j-1],:]))
        emission[j] = (np.random.choice(D,None, True, O[states[j],:]))
        syl_count= syl_dict[obs_map_r[emission[j]]]
        end_count = [x for x in syl_count if x.startswith('E')]
        if len(end_count) >= 1:
          syl_count=int(end_count[0][-1])
        else:
          syl_count = int(syl_count[0])
    try:
      k+=syl_count # or just pick the first one
    except:
      import pdb; pdb.set_trace()
    #print(k)
    if k>=10:
      line_syls.append(k)
  sentence = [obs_map_r[i] for i in emission]
  print(' '.join(sentence).capitalize())
  # break the line after 4/8/12 lines
  if (i == 3 or i == 7 or i == 11):
    print('\n')
print(line_syls)

Strumpeted disgrace and it might thou foul
Expired prey suffer tired which time but should
Walks issueless their false of of conquest
Of of wards to should to toil outward that


Mine being it blest seem quick threescore and say
Thy angel prove to were out false hath not
Who betraying rotten being the fair and
Deep not triumph of my praise day my some


Who amis some lives my but could by did
Selfexample hath think it affections
Eye did are shore by filching beauty sin
Thee on what brief active made but not then


Virtue commence and richer of thy thou
Winter and but that their till doth for but
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


# Rhyming HMM

Loading the rhyming dictionary made using the example rhymes in the training data

In [34]:
# Loading from our github
rhyme_array_raw = np.genfromtxt('https://raw.githubusercontent.com/treyra/CS155_miniproject3/main/data/rhyme_list.txt',delimiter='\t',dtype=None,encoding='ISO-8859-1')

rhyme_array = np.zeros((len(rhyme_array_raw),2),dtype=object)
for i,rhymes in enumerate(rhyme_array_raw):
    #Stored as a np array written to a text file, so remove the brackets
    rhyme_array[i,0] = rhymes.split()[0][2:-1]
    rhyme_array[i,1] = rhymes.split()[1][1:-2]

In [62]:
class RhymingHiddenMarkovModel ( HiddenMarkovModel ):
    '''
    Class implementation of Hidden Markov Models for rhyming sonnets
    '''

    def __init__(self, A, O,rhyme_array,syl_dict,obs_map):
        
        self.rhyme_array = rhyme_array
        self.syl_dict = syl_dict
        self.obs_map = obs_map
        self.obs_map_r = obs_map_reverser(obs_map)
        HiddenMarkovModel.__init__(self,A,O)

    def emit_line(self, numSyls = 10, endSeed=None):
        """
        Generate a random line backwards to forwards
        using a seeded ending word, if provided
        Parameters
        ----------
        numSyls : int
            Number of syllables the line should have. 
            Default is 10
        endSeed : tuple (int,string)
            The HMM state and the string the line will
            end in, if any.
        """
        #Line to emit
        line = None 

        #No seed, pick random initial state
        if endSeed is None:
            state = np.random.randint(0,self.L)
            line = np.array([np.random.choice(self.D,p=self.O[state,:])])
        else:
            state = endSeed[0]
            line = np.array([self.obs_map[endSeed[1]]])
        k = 0
        # Initial state
        syl_count= syl_dict[obs_map_r[line[0]]]
        if len(syl_count)>1:
          # check to use the one not at the end or just randomly
          not_end_count = [x for x in syl_count if not x.startswith('E')]
          syl_count = not_end_count
          # if syl_count[0][0]!='E':
          #   syl_count = syl_count[0][0]
        try:
          k+=int(syl_count[0])
        except:
          import pdb; pdb.set_trace()
        #Now loop backwards and generate rest of line before word
        j = 0
        prevState = state
        while k < 10:
            j+=1
            # generate next word (NOTE: we flipped the A matrix because we want to know the 
            #probability of transitioning TO the previous state)
            state = np.random.choice(self.L,p= self.A[:,prevState]/np.linalg.norm(self.A[:,prevState],1))
            line = np.concatenate(([np.random.choice(self.D,p=self.O[state,:])],line)) 
            syl_count= syl_dict[obs_map_r[line[0]]]
            #print(obs_map_r[line[0]])
            if len(syl_count)>1:
              end_count = [x for x in syl_count if x.startswith('E')]
              if len(end_count) >= 1:
                if k + int(end_count[0][-1]) > 10:
                  # if equal to 10 pick this word and save sentence
                  if k + int(end_count[0][-1]) == 10:
                    syl_count=int(end_count[0][-1])
                # check to use the correct one
                else:
                  for s in syl_count:
                    if s[0]!='E':
                      syl_count = int(s)
              else:
                syl_count = int(syl_count[0])
            else:
              syl_count = int(syl_count[0])
            if k+syl_count>10:
              # regernerate
              while k+syl_count>10:
                state = np.random.choice(self.L,p= self.A[:,prevState]/np.linalg.norm(self.A[:,prevState],1))
                line[0] = (np.random.choice(self.D,p = self.O[state,:]))
                syl_count= syl_dict[obs_map_r[line[0]]]
                end_count = [x for x in syl_count if x.startswith('E')]
                if len(end_count) >= 1:
                  syl_count=int(end_count[0][-1])
                else:
                  syl_count = int(syl_count[0])
            try:
              k+=syl_count # or just pick the first one
            except:
              import pdb; pdb.set_trace()
            #print(k)
            if k>=10:
              return [obs_map_r[i] for i in line]
    def genRhymingSonnet(self):
        """
        Uses rhyme dict to seed lines for a rhyming sonnet
        """
        sonnet = np.array([[]],dtype=object)
        #Generate 14 rhyming lines
        #List of indicies already picked and the word in the pair used
        #(set to -1 initially)
        used = np.zeros((7,2)) - 1

        for i in range(14):
            #Let the first word in the rhyme pair be in any order
            firstWord = -1
            if i == 0 or i == 1 or i == 4 or i == 5 or i == 8 or i == 9 or i == 12:
                rhyme = np.random.randint(0,len(self.rhyme_array))
                #Redraw for uniqueness (desired behavior?)
                while rhyme in used:
                    rhyme = np.random.randint(0,len(self.rhyme_array))
                #Using integer math to get the right index for each pair of rhymes
                used[int((i+1)/2),0] = rhyme
                #Randomly draw the last word, save so we use other next time
                lastWord = np.random.randint(0,len(self.rhyme_array[rhyme]))
                used[int((i+1)/2),1] = lastWord
            else:
                #Using integer math to get the right index for each pair of rhymes
                rhyme = int(used[int((i-1)/2),0])
                #Get the other word in pair (this is to allow for N lenght lists of rhymes)
                lastWord = np.random.randint(0,len(self.rhyme_array[rhyme])-1)
                #If we got the word we already used, just assign the one we excluded from the draw
                if lastWord == used[int((i-1)/2),1]:
                    lastWord = len(self.rhyme_array[rhyme])-1
                lastWord = int(lastWord)
            line = self.emit_line(endSeed=(np.random.randint(0,self.L),self.rhyme_array[rhyme,lastWord]))
            print(i,end = ": ")
            print(' '.join(line).capitalize())
            sonnet = np.append(sonnet,line)           
        return sonnet


In [37]:
# train a HMM (only a few epochs)
betterHMM = unsupervised_HMM(obs, 20, 50)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50



In [38]:
# mount drive
# save data by line and data by sonnet
from google.colab import drive
drive.mount('./drive',force_remount=True)

Mounted at ./drive


In [39]:
np.save('/content/drive/My Drive/CS155_miniproject_3/20_50_A.npy', betterHMM.A)    
np.save('/content/drive/My Drive/CS155_miniproject_3/20_50_O.npy', betterHMM.O)    

In [42]:
#Gen the rhyming Sonnet
rhyme_hmm = RhymingHiddenMarkovModel(hmm8.A, hmm8.O,rhyme_array,syl_dict,obs_map)
sonnet = rhyme_hmm.genRhymingSonnet()

0: Desire of the shadow in cheeks a are
1: Five one lack she thy yore moods self love stand
2: Which and my up hand of yet let sun care
3: Thee marriage if lies this grows this said land
4: Than my to mortal hath friends since to need
5: That on and none my her making your hope
6: Limbs knew name store active anchored exceed
7: Times twofold back prime cures the at the scope
8: Keeps taught lovst like my another thy white
9: She may favour hate day and name alone
10: Thee darling though o depends a delight
11: Change or epitaph see eves leave truth prone
12: Thing is wit second methinks receivest
13: Pleasing is be not thou i deceivest


In [52]:
#Gen the rhyming Sonnet
rhyme_hmm = RhymingHiddenMarkovModel(betterHMM.A, betterHMM.O,rhyme_array,syl_dict,obs_map)
sonnet = rhyme_hmm.genRhymingSonnet()

0: Policy whilst every being poor feeding
1: Or i daily is i my wantonly
2: Self by persuade your hours which right needing
3: Woe hems shame mayst find can beauty love dye
4: Even watching waste heart time pity for pride
5: And and in till boast i in proceeds ill
6: One bids answers why kingdom she break ride
7: In in thou be thou filching in in skill
8: Hours time might if may any might left wronk
9: By in thou in suit thou mercy and moan
10: And were out what man i my my nights tongue
11: Last this his still think the love this the gone
12: Live worth she woe and griefs your tell disgrace
13: For thou grace a five your thy beautys face


In [53]:
# train a HMM (only a few epochs)
betterHMM = unsupervised_HMM(obs, 15, 50)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50



In [54]:
np.save('/content/drive/My Drive/CS155_miniproject_3/15_50_A.npy', betterHMM.A)    
np.save('/content/drive/My Drive/CS155_miniproject_3/15_50_O.npy', betterHMM.O)    

In [55]:
# train a HMM (only a few epochs)
betterHMM = unsupervised_HMM(obs, 10, 50)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50



In [56]:
np.save('/content/drive/My Drive/CS155_miniproject_3/10_50_A.npy', betterHMM.A)    
np.save('/content/drive/My Drive/CS155_miniproject_3/10_50_O.npy', betterHMM.O)    

In [57]:
# train a HMM (only a few epochs)
hmm5 = unsupervised_HMM(obs, 5, 50)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50



In [58]:
np.save('/content/drive/My Drive/CS155_miniproject_3/5_50_A.npy', hmm5.A)    
np.save('/content/drive/My Drive/CS155_miniproject_3/5_50_O.npy', hmm5.O)    

In [60]:
#Gen the rhyming Sonnet
rhyme_hmm = RhymingHiddenMarkovModel(betterHMM.A, betterHMM.O,rhyme_array,syl_dict,obs_map)
sonnet = rhyme_hmm.genRhymingSonnet()

0: Tongue beauties lays out proud my lips taken
1: Leads thou how true birds cease back assured
2: Inward the a for pale sweet keep shaken
3: Now that all thine hath hath if and endured
4: Tan thee i that why fire is that for stage
5: I must form i despite those do then verse
6: Dulness forgive friend fortune absence rage
7: Is many the from i desire birds rehearse
8: True thy suns but i victor doth behind
9: Mistress in they then even of believe style
10: Worlds finds sweet gravity these how since kind
11: My consumst my rigour form kind compile
12: Height the is a a seeming house my know
13: Needs then so state edge which i gain eyes show


In [63]:
#Gen the rhyming Sonnet
rhyme_hmm = RhymingHiddenMarkovModel(hmm5.A, hmm5.O,rhyme_array,syl_dict,obs_map)
sonnet = rhyme_hmm.genRhymingSonnet()

0: Whom from invention some poor with a thee
1: Who say where and and which not largess love
2: Which be for i red and i pleasure me
3: Rare touches from wet is on my in prove
4: That confounds happy as looks for i thine
5: Through mind love fears fortune speak ordering
6: Should and alone have but enough to mine
7: My his on at preposterously in sing
8: And and folly hate untutored it edge
9: Poor sum long high remembered high as dyed
10: Red you being would as although privilege
11: To a is wombs his all to dignified
12: Who his the sins loves thy my not swearing
13: Swear haste but beauty rich ever bearing


#RNN model